In [147]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

Fase 1: Exploración y Limpieza

1.Exploración Inicial:
Realiza una exploración inicial de los datos para identificarposibles problemas, como valores nulos, atípicos o datosfaltantes en las columnas relevantes.
Utiliza funciones de Pandas para obtener informaciónsobre la estructura de los datos, la presencia de valoresnulos y estadísticas básicas de las columnas involucradas.
Une los dos conjuntos de datos de la forma más eficiente.

In [148]:
df_CLH = pd.read_csv("Customer Loyalty History.csv", index_col=None)
df_CFA = pd.read_csv("Customer Flight Activity.csv", index_col=None)

In [149]:
# Función para exploración general de datos
def exploracion_datos(df):
    print('_____________ INFORMACIÓN GENERAL DEL DATAFRAME ____________\n')
    print(df.info())
    
    print('___________________ FORMA DEL DATAFRAME ____________________\n')
    print(f"El número de filas que tenemos es de {df.shape[0]}.\nEl número de columnas es de {df.shape[1]}\n")
    
    print('_______________ NULOS, ÚNICOS Y DUPLICADOS _________________\n')
    print('La cantidad de valores NULOS por columna es de:\n')
    print(df.isnull().sum())
    print('____________________________________________________________\n')

    print('El porcentaje de valores NULOS por columna es de:\n')
    porcentaje_nulos = (df.isnull().sum() / df.shape[0]) * 100
    porcentaje_nulos = porcentaje_nulos.round(2)
    print(porcentaje_nulos)
    print('____________________________________________________________\n')

    print('La cantidad de valores ÚNICOS por columna es de:\n')
    for columna in df.columns:
        cantidad_unicos = df[columna].nunique()
        print(f'La columna {columna} tiene {cantidad_unicos} valores únicos.')
    print('____________________________________________________________\n')

    print('La cantidad de valores DUPLICADOS por columna es de:\n')
    for columna in df.columns:
        cantidad_duplicados = df.duplicated(subset=columna).sum()
        print(f'La columna {columna} tiene {cantidad_duplicados} valores duplicados.')
    print('____________________________________________________________\n')
    
    print('____________________ RESUMEN ESTADÍSTICO ____________________\n')
    print('___________________ Variables Numéricas _____________________\n')
    if df.select_dtypes(include=[float, int]).empty:
        print("No hay variables numéricas para mostrar el resumen estadístico.")
    else:
        print(df.describe().T)
    print('____________________________________________________________\n')
    print('___________________ Variables Categóricas _____________________\n')
    if df.select_dtypes(include=[object]).empty:
        print("No hay variables categóricas para mostrar el resumen estadístico.")
    else:
        print('__________________ Variables Categóricas ____________________\n')
        print(df.describe(include='object').T)

In [150]:
# Llamamos a la función para explorar el dataframe df_CLH 
exploracion_datos(df_CLH)

_____________ INFORMACIÓN GENERAL DEL DATAFRAME ____________

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16737 entries, 0 to 16736
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Loyalty Number      16737 non-null  int64  
 1   Country             16737 non-null  object 
 2   Province            16737 non-null  object 
 3   City                16737 non-null  object 
 4   Postal Code         16737 non-null  object 
 5   Gender              16737 non-null  object 
 6   Education           16737 non-null  object 
 7   Salary              12499 non-null  float64
 8   Marital Status      16737 non-null  object 
 9   Loyalty Card        16737 non-null  object 
 10  CLV                 16737 non-null  float64
 11  Enrollment Type     16737 non-null  object 
 12  Enrollment Year     16737 non-null  int64  
 13  Enrollment Month    16737 non-null  int64  
 14  Cancellation Year   2067 non-null   floa

In [151]:
# Vemos que aspecto tienen las 5 primeras columnas del dataframe df_CLH 
df_CLH.head()

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
0,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN
1,549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,NaN,Divorced,Star,3839.61,Standard,2016,3,NaN,NaN
2,429460,Canada,British Columbia,Vancouver,V6E 3D9,Male,College,NaN,Single,Star,3839.75,Standard,2014,7,2018.0,1.0
3,608370,Canada,Ontario,Toronto,P1W 1K4,Male,College,NaN,Single,Star,3839.75,Standard,2013,2,NaN,NaN
4,530508,Canada,Quebec,Hull,J8Y 3Z5,Male,Bachelor,103495.0,Married,Star,3842.79,Standard,2014,10,NaN,NaN


In [152]:
# Llamamos a la función para explorar el dataframe df_CFA
exploracion_datos(df_CFA)

_____________ INFORMACIÓN GENERAL DEL DATAFRAME ____________

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405624 entries, 0 to 405623
Data columns (total 10 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Loyalty Number               405624 non-null  int64  
 1   Year                         405624 non-null  int64  
 2   Month                        405624 non-null  int64  
 3   Flights Booked               405624 non-null  int64  
 4   Flights with Companions      405624 non-null  int64  
 5   Total Flights                405624 non-null  int64  
 6   Distance                     405624 non-null  int64  
 7   Points Accumulated           405624 non-null  float64
 8   Points Redeemed              405624 non-null  int64  
 9   Dollar Cost Points Redeemed  405624 non-null  int64  
dtypes: float64(1), int64(9)
memory usage: 30.9 MB
None
___________________ FORMA DEL DATAFRAME ____________________

E

Loyalty Number                 0
Year                           0
Month                          0
Flights Booked                 0
Flights with Companions        0
Total Flights                  0
Distance                       0
Points Accumulated             0
Points Redeemed                0
Dollar Cost Points Redeemed    0
dtype: int64
____________________________________________________________

El porcentaje de valores NULOS por columna es de:

Loyalty Number                 0.0
Year                           0.0
Month                          0.0
Flights Booked                 0.0
Flights with Companions        0.0
Total Flights                  0.0
Distance                       0.0
Points Accumulated             0.0
Points Redeemed                0.0
Dollar Cost Points Redeemed    0.0
dtype: float64
____________________________________________________________

La cantidad de valores ÚNICOS por columna es de:

La columna Loyalty Number tiene 16737 valores únicos.
La columna Ye

In [153]:
df_CFA.head()

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
0,100018,2017,1,3,0,3,1521,152.0,0,0
1,100102,2017,1,10,4,14,2030,203.0,0,0
2,100140,2017,1,6,0,6,1200,120.0,0,0
3,100214,2017,1,0,0,0,0,0.0,0,0
4,100272,2017,1,0,0,0,0,0.0,0,0


In [154]:
# Unión de los dataframes 
merged_df = pd.merge(df_CLH, df_CFA, on='Loyalty Number', how='inner')

In [155]:
# Comprobamos que se han unido correctamente
exploracion_datos(merged_df)

_____________ INFORMACIÓN GENERAL DEL DATAFRAME ____________

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405624 entries, 0 to 405623
Data columns (total 25 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Loyalty Number               405624 non-null  int64  
 1   Country                      405624 non-null  object 
 2   Province                     405624 non-null  object 
 3   City                         405624 non-null  object 
 4   Postal Code                  405624 non-null  object 
 5   Gender                       405624 non-null  object 
 6   Education                    405624 non-null  object 
 7   Salary                       302952 non-null  float64
 8   Marital Status               405624 non-null  object 
 9   Loyalty Card                 405624 non-null  object 
 10  CLV                          405624 non-null  float64
 11  Enrollment Type              405624 non-null  object 
 

In [156]:
merged_df.sample(5)

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
278241,532978,Canada,Ontario,Sudbury,M5V 1G5,Male,High School or Below,80604.0,Married,Star,2648.35,Standard,2014,10,NaN,NaN,2017,10,0,0,0,0,0.0,0,0
116705,819521,Canada,British Columbia,Victoria,V10 6T5,Female,Master,69772.0,Divorced,Star,10083.49,2018 Promotion,2018,3,2018.0,11.0,2018,6,16,7,23,5037,503.0,0,0
263281,197067,Canada,Ontario,London,M5B 3E4,Male,High School or Below,55218.0,Married,Star,2511.94,Standard,2014,11,NaN,NaN,2017,4,10,5,15,3750,375.0,375,30
266867,597294,Canada,Ontario,Ottawa,K1F 2R2,Female,Bachelor,52971.0,Married,Star,2791.88,2018 Promotion,2018,2,NaN,NaN,2017,12,0,0,0,0,0.0,0,0
389139,596818,Canada,Ontario,Toronto,M2M 7K8,Female,Bachelor,96926.0,Single,Star,14042.10,Standard,2013,10,NaN,NaN,2017,4,4,0,4,464,46.0,0,0


Fase 1: Exploración y Limpieza

2.Limpieza de Datos:
Elimina o trata los valores nulos, si los hay, en lascolumnas clave para asegurar que los datos esténcompletos.
Verifica la consistencia y corrección de los datos paraasegurarte de que los datos se presenten de formacoherente.
Realiza cualquier ajuste o conversión necesaria en lascolumnas (por ejemplo, cambiar tipos de datos) paragarantizar la adecuación de los datos para el análisisestadístico.

In [157]:
# Eliminamos las columnas "Cancelation Year" y "Cancelation Month" porque tienen un altgo porcentaje de nulos y no son relevantes para nuestro análisis. También "Country" porque es el mismo para todos los registros.
merged_df = merged_df.drop(columns=['Cancellation Year', 'Cancellation Month', 'Country'])

# Verificamos las columnas después de eliminar
print("Columnas después de eliminar:")
print(merged_df.columns)

Columnas después de eliminar:
Index(['Loyalty Number', 'Province', 'City', 'Postal Code', 'Gender',
       'Education', 'Salary', 'Marital Status', 'Loyalty Card', 'CLV',
       'Enrollment Type', 'Enrollment Year', 'Enrollment Month', 'Year',
       'Month', 'Flights Booked', 'Flights with Companions', 'Total Flights',
       'Distance', 'Points Accumulated', 'Points Redeemed',
       'Dollar Cost Points Redeemed'],
      dtype='object')


In [158]:
# Hacemos función para comparar la media y la mediana de una variable. En una distribución simétrica y sin valores atípicos, la media y la mediana deberían ser aproximadamente iguales.
def cal_min_median(df, columna):
    media = df[columna].mean()
    mediana = df[columna].median()
    return media, mediana

In [159]:
# Comparamos la media y la mediana de la columna "Salary" para ver si imputamos sus nulos con la media o la mediana. 
cal_min_median(merged_df, "Salary")

(79268.82595262615, 73479.0)

In [160]:
# Función para imputar los nulos con la mediana. 
def null_to_median(df, columna):
    mediana = df[columna].median()
    df[columna] = df[columna].fillna(mediana)
    return df

In [161]:
# Como la media y la mediana son bastante diferentes vamos a optar imputar con la mediana. 
null_to_median(merged_df, "Salary")

# Comprobamos si se ha hecho correctamente
merged_df['Salary'].isnull().sum()

0

In [162]:
# Función para pasar float a integer en el caso de la columna "Salary" 
def float_to_int(df, columns):
    for col in columns:
        df[col] = df[col].astype(int)
    
    return df

In [163]:
# Pasamos el tipo de dato de float a integer en el caso de "Salary" porque no los valores de esta columna no tienen decimales
col_float_to_int = ["Salary"]
float_to_int(merged_df, col_float_to_int)

# Comprobamos que se ha hecho correctamente
merged_df['Salary'].dtype

dtype('int32')

In [164]:
# Diccionario para mapear números de mes a nombres de mes
month_dict = {
    1: 'Enero',
    2: 'Febrero',
    3: 'Marzo',
    4: 'Abril',
    5: 'Mayo',
    6: 'Junio',
    7: 'Julio',
    8: 'Agosto',
    9: 'Septiembre',
    10: 'Octubre',
    11: 'Noviembre',
    12: 'Diciembre'
    }

# Convertimos los números de mes a nombres de mes utilizando map 
merged_df['Month'] = merged_df['Month'].map(month_dict)

# Vemos si se ha hecho correctamente
merged_df.head()

,Loyalty Number,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
0,480934,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236,Married,Star,3839.14,Standard,2016,2,2017,Enero,0,0,0,0,0.0,0,0
1,480934,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236,Married,Star,3839.14,Standard,2016,2,2017,Febrero,3,0,3,2823,282.0,0,0
2,480934,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236,Married,Star,3839.14,Standard,2016,2,2017,Marzo,0,0,0,0,0.0,0,0
3,480934,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236,Married,Star,3839.14,Standard,2016,2,2017,Abril,0,0,0,0,0.0,0,0
4,480934,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236,Married,Star,3839.14,Standard,2016,2,2018,Octubre,6,2,8,3352,335.0,465,38


In [165]:
# Función para eliminar regkistros (filas) duplicadas
def drop_dup(df):
    # Verificar filas duplicadas basadas en todas las columnas
    dup_rows = df.duplicated(keep=False)

    # Filtrar las filas duplicadas
    dup = df[dup_rows]

    # Eliminar todas las filas duplicadas
    no_dup_df = df.drop_duplicates(keep=False)

    return no_dup_df, dup

In [166]:
# Llamamos a la función, que nos devuelve dos dataframes (por si acaso), el que tiene y el que no tiene duplicados 
no_dup_df, dup = drop_dup(merged_df)

In [167]:
# El dataframe sin duplicados tiene menos registros, así que las duplicidades se han eliminado correctamente. A partir de ahora usaremos este dataframe para aplicar lass siguientes acciones. 
no_dup_df.shape

(401912, 22)

In [168]:
# Función para chequear si hay valores negativos en las columnas numéricas
def neg_values(df):
    neg_counts = {}

    for column in df.columns:
        if pd.api.types.is_numeric_dtype(df[column]):
            neg_counts[column] = (df[column] < 0).sum()

    return neg_counts


# Llamar a la función para contar valores negativos
neg_counts = neg_values(no_dup_df)

# Mostrar los resultados
print("Número de valores negativos por columna:")
for column, count in neg_counts.items():
    print(f"Columna '{column}': {count} valores negativos")

Número de valores negativos por columna:
Columna 'Loyalty Number': 0 valores negativos
Columna 'Salary': 480 valores negativos
Columna 'CLV': 0 valores negativos
Columna 'Enrollment Year': 0 valores negativos
Columna 'Enrollment Month': 0 valores negativos
Columna 'Year': 0 valores negativos
Columna 'Flights Booked': 0 valores negativos
Columna 'Flights with Companions': 0 valores negativos
Columna 'Total Flights': 0 valores negativos
Columna 'Distance': 0 valores negativos
Columna 'Points Accumulated': 0 valores negativos
Columna 'Points Redeemed': 0 valores negativos
Columna 'Dollar Cost Points Redeemed': 0 valores negativos


In [169]:
#Sustituimos valores negativos por positivos
no_dup_df.loc[no_dup_df['Salary'] < 0, 'Salary'] = no_dup_df['Salary'].abs()

#Comprobamos que ya no hay valores negativos
no_dup_df[no_dup_df['Salary'] < 0]

,Loyalty Number,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed


Fase 2: Visualización

Usando las herramientas de visualización que has aprendido duranteeste módulo, contesta a las siguientes gráficas usando la mejor gráficaque consideres:

1.¿Cómo se distribuye la cantidad de vuelos reservados pormes durante el año?

2.¿Existe una relación entre la distancia de los vuelos y lospuntos acumulados por los clientes?

3.¿Cuál es la distribución de los clientes por provincia oestado?

4.¿Cómo se compara el salario promedio entre los diferentesniveles educativos de los clientes?

5.¿Cuál es la proporción de clientes con diferentes tipos detarjetas de fidelidad?

6.¿Cómo se distribuyen los clientes según su estado civil y género?